In [ ]:
from finlab import data

#guide can be found here
#https://mrjbq7.github.io/ta-lib/funcs.html
#https://twopirllc.github.io/pandas-ta/#indicators-by-category


In [ ]:
from finlab import data
import finlab
from finlab.backtest import sim

finlab.login(api_token="QNBAVxVcsFjg0vZ5Um8P4XVtRWnhnMtznBpGjGHXqQ+vBYWkN6kXnXDrW70KhH4v#free")

In [ ]:
from finlab import backtest

In [ ]:
!pip install 

In [ ]:
!brew install ta-lib

In [ ]:
from finlab import data
from finlab.backtest import sim

rev = data.get('monthly_revenue:當月營收')
rev_ma3 = rev.average(3)
rev_ma12 = rev.average(12)

# 營收趨勢多頭策略
cond1 = rev_ma3/rev_ma12 > 1.1
cond2 = rev/rev.shift(1) > 0.9
cond_all = cond1 & cond2

pe = data.get('price_earning_ratio:本益比')
營業利益成長率 = data.get('fundamental_features:營業利益成長率')

# 本益成長比
peg = (pe/營業利益成長率)

# 本益成長比和原訊號相乘，若不持有則相乘後等於0
position = peg*(cond_all)

# 原訊號為0的不要選，若沒加這行且策略只選到7檔，之後還是會選3檔訊號為0(不持有)的補足10檔，執行這行就會排除訊號為0。
position = position[position>0]

# 選股挑本益成長比前10小的
position = position.is_smallest(10)

In [ ]:
print("hello world")
from finlab import data
from finlab.backtest import sim
foreign_buy=data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')
foreign_cont=(foreign_buy>0).sustain(2)

# 選出 RSI 最大的 20 檔股票
rsi = data.indicator('RSI')
position = rsi.is_largest(20)

# 回測，每週(W)調整一次
report = sim(position, resample='W', name="高RSI策略")
report.display()

In [ ]:
!pip install talib

In [ ]:
print("hello world")
from finlab import data
from finlab.backtest import sim

股本 = data.get('financial_statement:股本')
price = data.get('price:收盤價')
市值 = 股本 * price / 10 * 1000

df1 = data.get('financial_statement:投資活動之淨現金流入_流出')
df2 = data.get('financial_statement:營業活動之淨現金流入_流出')
自由現金流 = (df1 + df2).rolling(4).mean()

稅後淨利 = data.get('fundamental_features:經常稅後淨利')
權益總計 = data.get('financial_statement:股東權益總額')
股東權益報酬率 = 稅後淨利/ 權益總計

營業利益成長率 = data.get('fundamental_features:營業利益成長率')

當月營收 = data.get('monthly_revenue:當月營收')* 1000
當季營收 = 當月營收.rolling(4).sum()
市值營收比 = 市值 / 當季營收
foreign_buy=data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)')
foreign_buy_t=foreign_buy.loc[foreign_buy.index[1:]]
foreign_buy_tm1=foreign_buy.shift(1).loc[foreign_buy.index[1:]]
foreign_buy_diff=foreign_buy_tm1-foreign_buy
cond6=foreign_buy>0

increase=data.get('internal_equity_changes:董監增加股數')

cond7=increase>0

rsi = data.indicator('RSI')
rsi_con=rsi.is_largest(50)


condition1 = (市值 >1e9)
condition2 = 自由現金流 > 0
condition3 = 股東權益報酬率 > 0
condition4 = 營業利益成長率 > 0
condition5 = 市值營收比 < 5

position = condition1& condition2 & condition3 & condition4 & condition5 &cond7
report = sim(position, resample='W', name="優等生策略")
report.display()

In [ ]:
increase

In [ ]:
from finlab.backtest import sim
from finlab import data

data.set_universe(market='TSE_OTC')

adj_close = data.get('etl:adj_close').loc['2012':]
ema_5=data.indicator('EMA', adjust_price=True, resample='D',timeperiod=5)
ema_10=data.indicator('EMA', adjust_price=True, resample='D',timeperiod=10)
ema_20=data.indicator('EMA', adjust_price=True, resample='D',timeperiod=20)
atr=data.indicator('ATR', adjust_price=True, resample='D',timeperiod=10)
keltner_up=ema_10+2*atr
vol=data.get('price:成交股數')/1000
vol_avg=vol.average(20)

cond1 = (adj_close > keltner_up) & (adj_close.shift() < keltner_up.shift())
cond2 = (ema_5 > ema_10) & (ema_10 > ema_20)
cond3 = vol_avg > 100
entries =  cond1 & cond2 & cond3
exits = adj_close < keltner_up

position = entries.hold_until(exits,nstocks_limit=20, rank=vol_avg)
rr=sim(position,resample='2W',position_limit=0.05)

In [ ]:
from finlab import data
from finlab.backtest import sim

pe = data.get('price_earning_ratio:本益比')
rev=data.get('monthly_revenue:當月營收')
rev_ma3=rev.average(3)
rev_ma12=rev.average(12)
營業利益成長率=data.get('fundamental_features:營業利益成長率').deadline()
peg=(pe/營業利益成長率)
cond1= rev_ma3/rev_ma12>1.1
cond2=rev/rev.shift(1)>0.9

cond_all=cond1 & cond2
result=peg*(cond_all)
position=result[result>0].is_smallest(10).reindex(rev.index_str_to_date().index, method='ffill')

sim(position=position, fee_ratio=1.425/1000/3,stop_loss=0.1,upload=True,name='本益成長比')

In [ ]:
from finlab import data
from finlab import backtest

收盤價 = data.get('price:收盤價')
股價淨值比 = data.get("price_earning_ratio:股價淨值比")

# 條件 1
收盤價大於五 = 收盤價 > 5

# 條件 2
季線 = 收盤價.rolling(60).mean()
收盤價在季線之上 = 收盤價 > 季線

# 條件 3
總條件 = (收盤價 * 股價淨值比)[收盤價大於五&收盤價在季線之上].is_smallest(20)


backtest.sim(總條件, resample='Q', live_performance_start='2021-01-01')

In [ ]:
from finlab import data
from finlab.backtest import sim

close=data.get('price:收盤價')
upperband, middleband, lowerband = data.indicator('BBANDS', resample='D', nbdevup=2.5, nbdevdn=2.5, timeperiod=40)
entries = lowerband > close
exits = close > middleband

position = entries.hold_until(exits,nstocks_limit=100000,take_profit=0.1,stop_loss=0.15)['2330']
report=sim(position, upload=True, mae_mfe_window=40, name='護國神山救難隊')

In [ ]:
from finlab.backtest import sim
close = data.get("price:收盤價")
position = (close == close.rolling(250).max())
report = sim(position, resample="M", name="創年新高策略",upload=False)
report.display()

In [ ]:
from finlab import data
from finlab.backtest import sim

# 下載 ROE 跟收盤價
roe = data.get("fundamental_features:ROE稅後")
close = data.get("price:收盤價")
position = ((close / close.shift(60)).is_largest(30) & (roe > 0))

# 回測，每月(M)調整一次
report = sim(position, resample="M", upload=False)
report.display()

In [ ]:
from finlab import data
from finlab.backtest import sim

rev = data.get('monthly_revenue:當月營收')
rev_ma3 = rev.average(3)
rev_ma12 = rev.average(12)

# 營收趨勢多頭策略
cond1 = rev_ma3/rev_ma12 > 1.1
cond2 = rev/rev.shift(1) > 0.9
cond_all = cond1 & cond2

pe = data.get('price_earning_ratio:本益比')
營業利益成長率 = data.get('fundamental_features:營業利益成長率')

# 本益成長比
peg = (pe/營業利益成長率)

# 本益成長比和原訊號相乘，若不持有則相乘後等於0
position = peg*(cond_all)

# 原訊號為0的不要選，若沒加這行且策略只選到7檔，之後還是會選3檔訊號為0(不持有)的補足10檔，執行這行就會排除訊號為0。
position = position[position>0]

# 選股挑本益成長比前10小的
position = position.is_smallest(10)

In [ ]:
from finlab import data
from finlab.backtest import sim

股本 = data.get('financial_statement:股本')
price = data.get('price:收盤價')
市值 = 股本 * price / 10 * 1000

df1 = data.get('financial_statement:投資活動之淨現金流入_流出')
df2 = data.get('financial_statement:營業活動之淨現金流入_流出')
自由現金流 = (df1 + df2).rolling(4).mean()

稅後淨利 = data.get('fundamental_features:經常稅後淨利')
權益總計 = data.get('financial_statement:股東權益總額')
股東權益報酬率 = 稅後淨利/ 權益總計

營業利益成長率 = data.get('fundamental_features:營業利益成長率')

當月營收 = data.get('monthly_revenue:當月營收')* 1000
當季營收 = 當月營收.rolling(4).sum()
市值營收比 = 市值 / 當季營收


condition1 = (市值 < 1e10)
condition2 = 自由現金流 > 0
condition3 = 股東權益報酬率 > 0
condition4 = 營業利益成長率 > 0
condition5 = 市值營收比 < 5

position = condition1 & condition2 & condition3 & condition4 & condition5
report = sim(position, resample='M', name="優等生策略", upload=False)
report.display()

In [ ]:
from finlab import data
from finlab import backtest

close = data.get('price:收盤價')

position = close < 0
position['2330'] = 0.5
position['1101'] = -0.5

report = backtest.sim(position, upload=False)
report.display()

In [ ]:
from finlab import data
from finlab import backtest

pb = data.get('price_earning_ratio:股價淨值比')

# 股價淨值比前20%低
position = pb < pb.quantile_row(0.2)

position /= position.sum(axis=1)

# 多單總部位70%
position *= 0.7

# 空單總部位30%
position['0050'] = -0.3

report=backtest.sim( position, upload=False, resample='Q')
report.display()

In [ ]:
from finlab import data
from finlab.backtest import sim
import pandas as pd

rev = data.get("monthly_revenue:當月營收")
rev_rf = data.get("monthly_revenue:去年同月增減(%)")
vol = data.get("price:成交股數")/1000

rev_recent_3 = rev.rolling(3).sum()
vol_avg = vol.average(10)
cond1 = (rev_recent_3/rev_recent_3.rolling(24, min_periods=12).max())==1

# 成交量條件，排除流動性過小
cond2 = vol_avg > 300
# 過濾全額交割股
full_cash_delivery_stock_filter = data.get("etl:full_cash_delivery_stock_filter")

cond_all = cond1 & cond2 & full_cash_delivery_stock_filter 
result = rev_rf*(cond_all)
position = result[result>0].is_largest(10).reindex(rev.index_str_to_date().index, method="ffill")
report = sim(position=position,stop_loss=0.3,position_limit=0.1, upload=False)
report.display()

In [ ]:
from finlab import data
from finlab.backtest import sim

營業現金流 = data.get("financial_statement:營業活動之淨現金流入_流出").deadline()
投資現金流 = data.get("financial_statement:投資活動之淨現金流入_流出").deadline()
融資現金流 = data.get("financial_statement:籌資活動之淨現金流入_流出").deadline()
roe = data.get("fundamental_features:ROE稅後").deadline()
close = data.get("price:收盤價")

position = (營業現金流 > 0) & (投資現金流 > 0) & (融資現金流 < 0) & (roe > 5) & (close > 10)
from finlab import data
from finlab.backtest import sim

營業現金流 = data.get("financial_statement:營業活動之淨現金流入_流出").deadline()
投資現金流 = data.get("financial_statement:投資活動之淨現金流入_流出").deadline()
融資現金流 = data.get("financial_statement:籌資活動之淨現金流入_流出").deadline()
roe = data.get("fundamental_features:ROE稅後").deadline()
close = data.get("price:收盤價")

position = (營業現金流 > 0) & (投資現金流 > 0) & (融資現金流 < 0) & (roe > 5) & (close > 10)

# 過濾 KY股
sc = data.get("security_categories")
position_col = position.columns
ky_filter = position_col[~position_col.isin(list(sc[sc["name"].str.contains("KY")]["stock_id"]))]
position = position[ky_filter]

report = sim(position,resample="M",name="現金流正數", upload=False)
report.display()

In [ ]:
from finlab import data
from finlab.backtest import sim
import numpy as np


# 市場多空排列家數
def ls_order_position(short=5,mid=10,long=30):
    close = data.get("price:收盤價")
    short_ma = close.average(short)
    mid_ma = close.average(mid)
    long_ma = close.average(long)
    long_order = (short_ma>=mid_ma) & (mid_ma>=long_ma)
    long_order = long_order.sum(1)
    short_order = (short_ma<mid_ma) & (mid_ma<long_ma)
    short_order = short_order.sum(1)
    entry = long_order > short_order
    cond = ~close.isna()
    position = cond & entry
    return position
close = data.get("price:收盤價")
sma20 = close.average(20)
rev = data.get('monthly_revenue:當月營收')

# 多頭排列
cond1 = (close > close.shift(20)) & (close > close.shift(60))

# 短期營收大於長期營收
cond2 = (rev.average(3) > rev.average(12)).sustain(3)

buy = cond1 & cond2
sell = sma20 < sma20.shift()

position = pd.DataFrame(np.nan, index=buy.index, columns=buy.columns)
position[buy] = 1
position[sell] = 0
position = position.ffill().fillna(0).astype(bool)
position = position & ls_order_position()
report = sim(position, resample="D", name="ls_order_position",upload=False)
report.display()

In [ ]:

from finlab import data
from finlab.backtest import sim

close=data.get('price:收盤價')
upperband, middleband, lowerband = data.indicator('BBANDS', resample='D', nbdevup=2.5, nbdevdn=2.5,timeperiod=40)
entries = lowerband > close
exits = close > middleband

# 設定賣出條件
position = entries.hold_until(exits)['2330']
report = sim(position,upload=False,mae_mfe_window=40, name='護國神山救難隊')

In [ ]:
from finlab import data
from finlab.backtest import sim

close = data.get('price:收盤價')
pb = data.get('price_earning_ratio:股價淨值比')  

sma20 = close.average(20)
sma60 = close.average(60)

entries = close > sma20
exits = close < sma60

position = entries.hold_until(exits, nstocks_limit=10, rank=-pb, takeprofit=0.4)
backtest_report = sim(position, upload=False)

In [ ]:
from finlab import data
from finlab.backtest import sim

close = data.get('price:收盤價')
pb = data.get('price_earning_ratio:股價淨值比')  

sma20 = close.average(20)
sma60 = close.average(60)

entries = close > sma20
exits = close < sma60

position = entries.hold_until(exits, nstocks_limit=10, rank=-pb, takeprofit=0.4)
backtest_report = sim(position, upload=False)

In [ ]:
from finlab import data
from finlab import backtest

# 只買入價格小於 6 元的股票
close = data.get('price:收盤價')
position = close < 6

# 回測，每月底(M)重新調整股票權重
report = backtest.sim(position, resample='M', name="價格小於6的股票")


In [ ]:
from finlab import data
from finlab.backtest import sim

# 選出 RSI 最大的 20 檔股票
rsi = data.indicator('RSI')
position = rsi.is_largest(20)

# 回測，每月(M)調整一次
report = sim(position, resample='W', name="高RSI策略")

In [ ]:
from finlab import data
from finlab.backtest import sim

# 下載 ROE 跟收盤價
roe = data.get("fundamental_features:ROE稅後")
close = data.get('price:收盤價')

position = (
      (close / close.shift(60)).is_largest(30) # 選出乖離率前 30 名的股票
    & (roe > 0) # 選出 ROE 大於 0 的股票
)

# 回測，每月(M)調整一次
report = sim(position, resample='M', name="乖離率和ROE濾網策略")

In [ ]:
df=data.get('price:收盤價')
# 限定範圍
df=df[(df.index>'2015')&(df.index<'2019')]

cond1=df>df.rolling(5).mean()
cond2=df>df.rolling(10,min_periods=5).mean()
cond3=df>df.rolling(20,min_periods=10).mean()
cond4=df>df.rolling(60,min_periods=40).mean()

position=(df*(cond1&cond2&cond3&cond4))
position=position[position>0]
position=position.is_largest(10)

In [ ]:
df=data.get('price:收盤價')
mean_20=df.rolling(20).mean()
std_value=df.rolling(20).std()

up=mean_20+std_value*2

# 收盤價剛站上布林通道上緣
cond1=(df>up)&(df.shift()<up)
cond2=(df>5)&(df<200)
position=(cond1&cond2)
position=position[position>0]
position=position.is_smallest(20)

In [ ]:
df=data.get('price:收盤價')
mean_20=df.rolling(20).mean()
std_value=df.rolling(20).std()

up=mean_20+std_value*2

# 收盤價剛站上布林通道上緣
cond1=(df>up)&(df.shift()<up)
cond2=(df>5)&(df<200)
position=(cond1&cond2)
position=position[position>0]
position=position.is_smallest(20)
close = data.get("price:收盤價")
sma20 = close.average(20)
rev = data.get('monthly_revenue:當月營收')

# 多頭排列
cond1 = (close > close.shift(20)) & (close > close.shift(60))

# 短期營收大於長期營收
cond2 = (rev.average(3) > rev.average(12)).sustain(3)

buy = cond1 & cond2
sell = sma20 < sma20.shift()

position = pd.DataFrame(np.nan, index=buy.index, columns=buy.columns)
position[buy] = 1
position[sell] = 0
position = position.ffill().fillna(0).astype(bool)
position = position & ls_order_position()
report = sim(position, resample="D", name="ls_order_position",upload=False)
report.display()